In [258]:
#for if we want to export this notebook to LaTeX
!export PATH=/Library/TeX/texbin:$PATH

In [1]:
#bring in our standard imports 

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import time
import requests
import os
import rpy2


In this notebook, we are going to clean and prep all of the data so that it is a nice organized panel and ready to go for R !

# Original Data

In [260]:
tax_haven_data = \
    pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/variable data/tax_haven1.csv')

tax_haven_data.rename(columns = {
    'Year' : 'year', 
    'country' : 'country_name',
    'Number.of.foreign.corporations' : 'x_num_for_corps', 
    'Number.of.US.corporations.returns' : 'x_num_US_corps', 
    'Total.Assets' : 'x_total_assets', 
    'Total.Receipts' : 'x_total_receipts', 
    'Current.earnings.and.profits..less.deficit..before.taxes' : 'x_cur_EP', 
    'percent_Number.of.foreign.corporations' : 'x_p_num_for_corps', 
    'percent_Total.Assets' : 'x_p_total_assets', 
    'percent_Total.Receipts' : 'x_p_total_receipts', 
    'percent_Current.earnings.and.profits..less.deficit..before.taxes' : 'x_p_cur_EP'
}, inplace=True)

tax_haven_data.drop('Unnamed: 0',axis = 1, inplace=True)
tax_haven_data.sort_values(['year', 'country_code'], inplace=True)
tax_haven_data = tax_haven_data.set_index(['year', 'country_code'])


## Country Binaries

In [261]:
#create a binary country or region variable 

#list of country codes 
country_codes = pd.read_csv('https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv')
country_codes.head()

country_codes = country_codes['Alpha-3 code'].values
country_codes = [c[2:-1] for c in country_codes]

#go through everything and check 
tax_haven_data['c_country_binary'] = None 
nos = []


#if the computer isn't sure, as the user for input!
for year, country_code in tax_haven_data.index:
    
    country_name = tax_haven_data.loc[year].loc[country_code]['country_name']
    
    if country_code in country_codes: 
        tax_haven_data['c_country_binary'][year][country_code] = 1
        continue 
    
    else:
        if 'other' in country_name or 'total' in country_name or 'area' in country_name\
            or 'except' in country_name or 'countries' in country_name: 
            tax_haven_data['c_country_binary'][year][country_code] = 0
            continue 
            
        if country_code in nos:
            tax_haven_data['c_country_binary'][year][country_code] = 0
            continue 
            
        resp = input('\n Is ' + country_name + ' a country?')
        
        if resp == 'n':
            tax_haven_data['c_country_binary'][year][country_code] = 0
            nos.append(country_code)
        if resp == 'y':
            tax_haven_data['c_country_binary'][year][country_code] = 1
            country_codes.append(country_code)
            
         
    


 Is west germany a country?y

 Is europe a country?n

 Is zaire a country?n

 Is east germany a country?n
country_code
OTHER_WESTERNHEMISPHERE    \n Is other western hemisphere, total a country?
OTHER_WESTERNHEMISPHERE       \n Is all other western hemisphere a country?
Name: country_name, dtype: objectn
country_code
OTHER_WESTEUROPE    \n Is other west european countries, total a c...
OTHER_WESTEUROPE    \n Is all other west european countries a coun...
Name: country_name, dtype: objectn

 Is union of soviet socialist republics a country?n

 Is all geographic regions a country?n
country_code
OTHER_EUROPE    \n Is other european countries, total a country?
OTHER_EUROPE           \n Is other european countries a country?
Name: country_name, dtype: objectn


In [262]:
#filter down to only get items which are countries 
tax_haven_data = tax_haven_data.loc[(tax_haven_data['c_country_binary'] == 1)]

In [263]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP c_country_binary  
year country_code                               
1980 ANT             0.009460                1  
     AUS             0.032672                1  
     AUT             0.003303                1  
     BEL             0.000028                1  
     BHS             0.006778                1

# Controls Data

## Regional

In [264]:
regional = \
pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/controls data/regional_dummies.csv')
regional.head()

f_counter, s_counter = 0, 0

tax_haven_data['c_region'] = None
tax_haven_data['c_sub_region'] = None

for year, country_code in tax_haven_data.index:
    try:
        region = regional.loc[(regional['alpha-3'] == country_code)]['region'].values[0]
        sub_region = regional.loc[(regional['alpha-3'] == country_code)]['sub-region'].values[0]
        
        tax_haven_data['c_region'][year][country_code] = region
        tax_haven_data['c_sub_region'][year][country_code] = sub_region
        s_counter += 1
    except: 
        f_counter += 1 
print('Finished F S', f_counter, s_counter)

Finished F S 51 1054


In [265]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP c_country_binary  c_region  \
year country_code                                          
1980 ANT             0.009460                1      None   
     AUS             0.032672                1   Oceania   
     AUT             0.003303                1    Europe   
     BEL             0.000028                1    Europe   
     BHS             0.006778                1  Americas   

                                      c_sub_region  
year country_code                                   
1980 ANT                                      None  
     AUS                 Australia and New Zealand  
     AUT                            Western Europe  
     BEL                            Western Europe  
     BHS           Latin America and the Caribbean

## Land

In [266]:
land = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/controls data/land.csv')
land.set_index('Country Code', inplace=True)
tax_haven_data['c_land'] = None 

f_counter, s_counter = 0, 0

for year, country_code in tax_haven_data.index: 
    try:
        data_to_fill = land[str(year)][country_code]
        tax_haven_data['c_land'][year][country_code] = data_to_fill
        s_counter += 1
    except: 
        f_counter += 1
    
print('Finished F S', f_counter, s_counter)

Finished F S 39 1066


In [267]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP c_country_binary  c_region  \
year country_code                                          
1980 ANT             0.009460                1      None   
     AUS             0.032672                1   Oceania   
     AUT             0.003303                1    Europe   
     BEL             0.000028                1    Europe   
     BHS             0.006778                1  Americas   

                                      c_sub_region     c_land  
year country_code                                              
1980 ANT                                      None       None  
     AUS                 Australia and New Zealand  7682300.0  
     AUT                            Western Europe    82580.0  
     BEL                            Western Europe    32820.0  
     BHS           Latin America and the Caribbean    10010.0

## Resource Endowment

In [268]:
#we can now generalize how to add new WB data! 

def add_WB_data(tax_haven_data, new_df):
    s_counter, f_counter = 0,0 
    new_column = input('What would you like the new column to be?')
    tax_haven_data[new_column] = None 
    new_df.set_index('Country Code', inplace=True)
    
    for year, country_code in tax_haven_data.index: 
        try: 
            
            data_to_fill = new_df[str(year)][country_code]
            
            tax_haven_data[new_column][year][country_code] = data_to_fill
            s_counter += 1
        except: 
            f_counter += 1

    print('Finished S F', s_counter, f_counter)

In [269]:
resource_endowment = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/controls data/resource_endowmnet.csv')

add_WB_data(tax_haven_data, resource_endowment)


What would you like the new column to be?c_resource_endowment
Finished S F 1066 39


In [270]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP c_country_binary  c_region  \
year country_code                                          
1980 ANT             0.009460                1      None   
     AUS             0.032672                1   Oceania   
     AUT             0.003303                1    Europe   
     BEL             0.000028                1    Europe   
     BHS             0.006778                1  Americas   

                                      c_sub_region     c_land  \
year country_code                                               
1980 ANT                                      None       None   
     AUS                 Australia and New Zealand  7682300.0   
     AUT                            Western Europe    82580.0   
     BEL                            Western Europe    32820.0   
     BHS           Latin America and the Caribbean    10010.0   

                  c_resource_endowment  
year country_code                       
1980 ANT                          None  
     AUS                      6.267167  
     AUT                      0.637915  
     BEL                      0.137741  
     BHS                      0.226219

## Population


In [271]:
population = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/controls data/population.csv')

add_WB_data(tax_haven_data, population)

What would you like the new column to be?c_population
Finished S F 1066 39


In [272]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP c_country_binary  c_region  \
year country_code                                          
1980 ANT             0.009460                1      None   
     AUS             0.032672                1   Oceania   
     AUT             0.003303                1    Europe   
     BEL             0.000028                1    Europe   
     BHS             0.006778                1  Americas   

                                      c_sub_region     c_land  \
year country_code                                               
1980 ANT                                      None       None   
     AUS                 Australia and New Zealand  7682300.0   
     AUT                            Western Europe    82580.0   
     BEL                            Western Europe    32820.0   
     BHS           Latin America and the Caribbean    10010.0   

                  c_resource_endowment c_population  
year country_code                                    
1980 ANT                          None         None  
     AUS                      6.267167   14692000.0  
     AUT                      0.637915    7549433.0  
     BEL                      0.137741    9859242.0  
     BHS                      0.226219     210591.0

## Real GDP / Cap

In [273]:
def add_OWID(tax_haven_data, new_df): 
    s_counter, f_counter = 0, 0
    new_column = input('What would you like the new column to be?')
    tax_haven_data[new_column] = None
    var_of_interest = 'Variable'
    new_df.set_index(['Year', 'Code'], inplace = True)
    
    for year, country_code in tax_haven_data.index: 
        try: 
            data_to_fill = new_df[var_of_interest].loc[year].loc[country_code]
            
            tax_haven_data[new_column][year][country_code] = data_to_fill
            s_counter += 1
        except: 
            f_counter += 1
    print('Finished, F, S', f_counter, s_counter)

In [274]:
GDP = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/controls data/real_GDP_per_cap.csv')

add_OWID(tax_haven_data, GDP)

What would you like the new column to be?c_rGDP
Finished, F, S 61 1044


In [275]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP c_country_binary  c_region  \
year country_code                                          
1980 ANT             0.009460                1      None   
     AUS             0.032672                1   Oceania   
     AUT             0.003303                1    Europe   
     BEL             0.000028                1    Europe   
     BHS             0.006778                1  Americas   

                                      c_sub_region     c_land  \
year country_code                                               
1980 ANT                                      None       None   
     AUS                 Australia and New Zealand  7682300.0   
     AUT                            Western Europe    82580.0   
     BEL                            Western Europe    32820.0   
     BHS           Latin America and the Caribbean    10010.0   

                  c_resource_endowment c_population     c_rGDP  
year country_code                                               
1980 ANT                          None         None       None  
     AUS                      6.267167   14692000.0  22172.068  
     AUT                      0.637915    7549433.0  17684.906  
     BEL                      0.137741    9859242.0  20385.578  
     BHS                      0.226219     210591.0  23412.768

# Outcomes Data

We can automate this to be much faster! 

In [276]:
raw_outcomes_data = []

path = '/Users/zachmariani/Desktop/Metrics/Final Project/Data/outcomes data/'
for outcome_path in os.listdir(path):
    if outcome_path.split('.')[1] == 'csv': 
        
        full_path = path + outcome_path
        
        raw_outcomes_data.append((outcome_path, pd.read_csv(full_path)))

In [277]:
for path, raw_outcome in raw_outcomes_data: 
    print('This is: ', path)
    print('The variable should be y_')
    add_OWID(tax_haven_data, raw_outcome)

This is:  deaths_by_age.csv
The variable should be y_
What would you like the new column to be?y_deaths_under_14
Finished, F, S 366 739
This is:  central_government_expenditure_as_share_of_GDP.csv
The variable should be y_
What would you like the new column to be?y_gov_expend
Finished, F, S 353 752
This is:  HDI.csv
The variable should be y_
What would you like the new column to be?y_HDI
Finished, F, S 383 722
This is:  life_expectancy_at_birth.csv
The variable should be y_
What would you like the new column to be?y_life_expectancy
Finished, F, S 81 1024
This is:  homicides.csv
The variable should be y_
What would you like the new column to be?y_homicides
Finished, F, S 469 636


In [278]:
#replace any Nones with np.nans to stay uniform 
tax_haven_data.replace({None : np.nan}, inplace=True )

# Instrument Data 

## Corporate Income Tax Collected 

In [279]:
#our IV is total corporate income tax federal brought in by the US every year
#we will then need the importance term to get the interaction

IV = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/instrument data/US_tax_revenue.csv')
IV = IV.loc[(IV['Government'] == 'Total')].loc[(IV['Tax'] == 'Corporate income tax federal')]
IV.set_index('Year', inplace = True)
IV = IV[['Value']] 
IV.rename(columns = {'Value' : 'corp_income'}, inplace=True)
IV.head()

,corp_income
Year,
1965,24861.0
1966,29373.0
1967,33071.0
1968,27865.0
1969,35578.0


## Exposure to US / Importance

In [280]:
#get a cleaner mapping since the data isn't stored with Alpha-3 codes
country_codes = pd.read_csv('https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv')
country_codes.head()
cc_dict = country_codes.set_index('Country').to_dict()['Alpha-3 code']
cc_clean = {
    'british virgin islands' : 'VGB', 
    
}
for key, value in cc_dict.items():
    key = key.lower()
    value = value.split("\"")[1]
    cc_clean[key] = value

In [281]:
im_ex = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/instrument data/im_ex.csv')
im_ex = im_ex[['TOTAL', 'Exports', 'Imports']]
im_ex.rename(columns = {'TOTAL': 'country_name'}, inplace=True)
im_ex['country_name'] = im_ex['country_name'].str.lower()
im_ex = im_ex.replace('(-)', 0)
im_ex['country_name'] = im_ex['country_name'].str.replace('\*', '')

im_ex.set_index('country_name', inplace=True)
im_ex.rename(index = cc_clean, inplace=True)
im_ex.dropna(inplace = True)
im_ex['Total'] = im_ex['Exports'] + im_ex['Imports']
im_ex['Total/GDP'] = np.nan

im_ex.index.name = 'country_code'

for country_code in im_ex.index: 
    try:
        total_im_ex = im_ex.loc[country_code]['Total'] * 1000000000
        gdp_per_cap = GDP.loc[pd.IndexSlice[:, country_code], :]['Variable'].iloc[-1]
        pop = population['2017'].loc[country_code]
        gdp_total = gdp_per_cap * pop
        value = total_im_ex / gdp_total 
        im_ex['Total/GDP'][country_code] = value
    except: 
#         print('No data for', country_code)
        pass
im_ex.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Exports,Imports,Total,Total/GDP
country_code,,,,
AFG,57.9,2.3,60.2,NaN
ALB,3.8,3.9,7.7,0.224408
DZA,73.7,99.5,173.2,0.339566
AND,0.4,0.1,0.5,NaN
AGO,35.2,18.4,53.6,0.359973


## Interaction

The interaction term is just the IV term and the importance term multiplied 

In [307]:
tax_haven_data['IV_interaction'] = np.nan

for year, country_code in tax_haven_data.index: 
    
    try: 
        corp_income = IV.loc[year]['corp_income']

        importance = im_ex.loc[country_code]['Total/GDP']
        
        IV_interaction = corp_income * importance 
        
        tax_haven_data.at[(year, country_code), 'IV_interaction'] = IV_interaction
        
    except Exception as e : 
        pass

'ANT'
'WESTDEU'
'ANT'
'KOR'
'PRI'
'VIR'
'WESTDEU'
'ANT'
'KOR'
'PRI'
'VIR'
'WESTDEU'
'ANT'
'KOR'
'PRI'
'VIR'
'WESTDEU'
'ANT'
'ASM'
'GUM'
'IRN'
'KOR'
'PRI'
'TZA'
'VIR'
'WESTDEU'
'ANT'
'KOR'
'PRI'
'ANT'
'KOR'
'PRI'
'VIR'
'ANT'
'KOR'
'PRI'
'ANT'
'KOR'
'PRI'
'ANT'
'KOR'
'PRI'
'ANT'
'KOR'
'PRI'
'ANT'
'GGY'
'JEY'
'KOR'
'PRI'
'VIR'
'ANT'
'GGY'
'JEY'
'KOR'
'PRI'
'VIR'
'ANT'
'GGY'
'JEY'
'KOR'
'PRI'
'VIR'
'ANT'
'KOR'
'KOR'
'KOR'
'KOR'


In [308]:
tax_haven_data.head()

country_name  x_num_for_corps  x_num_US_corps  \
year country_code                                                         
1980 ANT           netherland antilles            345.0             260   
     AUS                     australia           1602.0             647   
     AUT                       austria            291.0             210   
     BEL                       belgium            835.0             540   
     BHS                       bahamas            260.0             146   

                   x_total_assets  x_total_receipts     x_cur_EP  \
year country_code                                                  
1980 ANT               14311863.0         2190820.0   450512.000   
     AUS               20418036.0        20639051.0  1555874.000   
     AUT                2488082.0         3847721.0   157297.000   
     BEL              143161653.0        22313903.0     1323.109   
     BHS                7524769.0        34951662.0   322780.000   

                   x_p_num_for_corps  x_p_total_assets  x_p_total_receipts  \
year country_code                                                            
1980 ANT                    0.009726          0.028171            0.003134   
     AUS                    0.045164          0.040190            0.029526   
     AUT                    0.008204          0.004897            0.005505   
     BEL                    0.023540          0.281797            0.031922   
     BHS                    0.007330          0.014812            0.050002   

                   x_p_cur_EP  ...     c_land c_resource_endowment  \
year country_code              ...                                   
1980 ANT             0.009460  ...        NaN                  NaN   
     AUS             0.032672  ...  7682300.0             6.267167   
     AUT             0.003303  ...    82580.0             0.637915   
     BEL             0.000028  ...    32820.0             0.137741   
     BHS             0.006778  ...    10010.0             0.226219   

                  c_population     c_rGDP  y_deaths_under_14  y_gov_expend  \
year country_code                                                            
1980 ANT                   NaN        NaN                NaN           NaN   
     AUS            14692000.0  22172.068                NaN     23.403527   
     AUT             7549433.0  17684.906                NaN     33.473942   
     BEL             9859242.0  20385.578                NaN           NaN   
     BHS              210591.0  23412.768                NaN     16.632966   

                   y_HDI  y_life_expectancy  y_homicides  IV_interaction  
year country_code                                                         
1980 ANT             NaN                NaN          NaN             NaN  
     AUS             NaN          74.333659         1.92    1.377656e+05  
     AUT           0.747          72.463415         1.21    1.627145e+05  
     BEL           0.755          73.207073         1.50    5.756995e+05  
     BHS             NaN          68.524000        11.42    1.817451e+06  

[5 rows x 23 columns]

# Lagging X Variables

In [ ]:
import statsmodels.api as sm

tax_haven_data = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/full_panel.csv')
tax_haven_data.head()

for column in tax_haven_data.columns: 
    if 'x_p_' in column: 
        tax_haven_data['shift1' + column] = tax_haven_data[column].shift(1)
        tax_haven_data['shift2' + column] = tax_haven_data[column].shift(2)



In [2]:
pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/full_panel_shifted.csv').columns

Index(['Unnamed: 0', 'year', 'country_code', 'country_name', 'x_num_for_corps',
       'x_num_US_corps', 'x_total_assets', 'x_total_receipts', 'x_cur_EP',
       'x_p_num_for_corps', 'x_p_total_assets', 'x_p_total_receipts',
       'x_p_cur_EP', 'c_country_binary', 'c_region', 'c_sub_region', 'c_land',
       'c_resource_endowment', 'c_population', 'c_rGDP', 'y_deaths_under_14',
       'y_gov_expend', 'y_HDI', 'y_life_expectancy', 'y_homicides',
       'IV_interaction', 'shift1x_p_num_for_corps', 'shift2x_p_num_for_corps',
       'shift1x_p_total_assets', 'shift2x_p_total_assets',
       'shift1x_p_total_receipts', 'shift2x_p_total_receipts',
       'shift1x_p_cur_EP', 'shift2x_p_cur_EP'],
      dtype='object')

# Export Cleaned Data!

In [7]:
tax_haven_data.to_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/full_panel_shifted.csv')

# Misc Notes

The IV is more a demand shock, it isolates US demands for tax havens. Threats are reverse causality, maybe incorporate US foreign investment. Flush out the problem that you think this would solve, what problems this still might have, if you can tell the story of what its solving and how you're solving it. 

You need some baseline measure of how exposed a country is the the US, you want a time invariant exposure proxy, some measure of trade/financial closeness to the US, this will give us the first stage. 

The IV then becomes the interaction between the time series US thing and the importance measure. 

Then add that as a column to the panel, should be instrument cluster by country, and regress outcome on instrument.

Check email for his paper, equations 1-4. For control variables you might forward fill, this is a little shady, you can fill in the missings and then have an indicator for if something is missing. 

Be clear on what problem the IV is solving and what it is not solving, pre-trend check and see if instrument predicts lag values of the outcome, should report first stage $f$ statistic, $X on Z$ and all the controls 

We should get an $f$ statistic of $10$ or more to avoid the weak instruments problem. 

You can check the journal's or the author's website to find their code. 

Also think about running the R code directly from the Python notebook. 

# Summary Statistics

In [10]:
tax_haven_data = pd.read_csv('/Users/zachmariani/Desktop/Metrics/Final Project/Data/full_panel.csv')
pd.set_option('display.max_columns', None)

In [12]:
summary = tax_haven_data.describe()

In [13]:
lst = []
for column in summary: 
    if 'x_p_' in column or 'year' in column or 'c_' in column or 'y_' in column: 
        lst.append(column)

In [24]:
try:
    summary = summary[lst]
    summary = summary.drop('c_country_binary', axis = 1)
except:
    pass
for column in summary: 
    summary[column] = round(summary[column], 2)

In [37]:
print(summary.iloc[:,8:].to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &     c\_rGDP &  y\_deaths\_under\_14 &  y\_gov\_expend &   y\_HDI &  y\_life\_expectancy &  y\_homicides \\
\midrule
count &    1044.00 &             739.00 &        752.00 &  722.00 &            1024.00 &       636.00 \\
mean  &   23910.36 &           80521.35 &         28.13 &    0.77 &              72.92 &         7.14 \\
std   &   41118.68 &          334702.25 &         11.92 &    0.12 &               7.58 &        12.54 \\
min   &     215.45 &               3.93 &          1.88 &    0.37 &              45.84 &         0.00 \\
25\%   &    7460.15 &             534.39 &         17.60 &    0.70 &              69.87 &         1.10 \\
50\%   &   18374.91 &            2585.43 &         27.51 &    0.80 &              74.62 &         1.90 \\
75\%   &   30417.73 &           16373.88 &         37.09 &    0.87 &              78.08 &         7.04 \\
max   &  536154.50 &         3173957.20 &         94.83 &    0.95 &              84.23 &        88.53 \\
\b

In [34]:
len(summary.columns)

14

In [ ]:
df.iloc[: , :N]